# Chapter 13: Responsible and Ethical AI  
## Bias, Fairness, Transparency, and Accountability

This notebook provides a hands-on simulation illustrating how algorithmic bias can arise,
how fairness metrics behave in practice, and how transparency tools help interpret AI decisions.

The goal is not to optimize performance, but to understand ethical risks and mitigation strategies
in AI systems used for real-world decision-making.


### Learning Objectives

By the end of this notebook, you will be able to:

- Identify how bias enters AI systems through data and modeling choices
- Evaluate fairness using common metrics (demographic parity, equal opportunity)
- Compare black-box and interpretable models
- Apply basic explainability tools to model predictions
- Connect technical outcomes to ethical, legal, and societal implications


### Simulation Scenario: Automated Hiring System

We simulate an AI system used to screen job applicants.
The model predicts whether an applicant is selected for an interview.

Features include:
- Years of experience
- Education level
- Test score
- Gender (protected attribute)

We will intentionally introduce bias to illustrate ethical risks.


In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix


In [2]:
np.random.seed(42)

n = 1000

gender = np.random.binomial(1, 0.5, n)  # 0 = Female, 1 = Male
experience = np.random.normal(5 + gender, 2, n)
education = np.random.binomial(1, 0.6, n)  # 1 = College degree
test_score = np.random.normal(70 + 5 * gender, 10, n)

# Introduce biased hiring rule
logit = (
    0.3 * experience +
    0.5 * education +
    0.04 * test_score +
    0.8 * gender  # explicit bias
)

prob = 1 / (1 + np.exp(-logit))
hired = np.random.binomial(1, prob)

df = pd.DataFrame({
    "gender": gender,
    "experience": experience,
    "education": education,
    "test_score": test_score,
    "hired": hired
})

df.head()


,gender,experience,education,test_score,hired
0,0,5.355402,0,82.374381,1
1,1,3.329311,1,70.426983,1
2,1,6.760396,1,74.571766,1
3,1,7.221171,1,75.580227,1
4,0,6.119581,0,78.484309,1


### Ethical Issue: Data and Model Bias

The outcome variable embeds bias by construction.
Gender directly influences hiring probability, even when qualifications are similar.

This mirrors real-world risks:
- Historical bias
- Proxy discrimination
- Unfair feature weighting


In [3]:
X = df[["experience", "education", "test_score", "gender"]]
y = df["hired"]

model = LogisticRegression()
model.fit(X, y)

preds = model.predict(X)

accuracy_score(y, preds)


0.99

### Fairness Evaluation

We evaluate whether the model treats demographic groups equitably.
We focus on:

- Demographic parity
- Equal opportunity


In [4]:
df["pred"] = preds

selection_rate_male = df[df.gender == 1]["pred"].mean()
selection_rate_female = df[df.gender == 0]["pred"].mean()

selection_rate_male, selection_rate_female


(np.float64(1.0), np.float64(1.0))

In [5]:
cm_male = confusion_matrix(
    df[df.gender == 1]["hired"],
    df[df.gender == 1]["pred"]
)

cm_female = confusion_matrix(
    df[df.gender == 0]["hired"],
    df[df.gender == 0]["pred"]
)

tpr_male = cm_male[1,1] / cm_male[1].sum()
tpr_female = cm_female[1,1] / cm_female[1].sum()

tpr_male, tpr_female


(np.float64(1.0), np.float64(1.0))

### Ethical Interpretation

Disparities in selection rates and true positive rates indicate unfair treatment.
Even with high accuracy, the system violates fairness principles.

This highlights why accuracy alone is insufficient in ethical AI evaluation.


In [6]:
X_no_gender = df[["experience", "education", "test_score"]]

model_fair = LogisticRegression()
model_fair.fit(X_no_gender, y)

preds_fair = model_fair.predict(X_no_gender)
df["pred_fair"] = preds_fair


In [7]:
df.groupby("gender")["pred_fair"].mean()


,pred_fair
gender,
0,1.0
1,1.0


### Explainable AI (XAI)

Transparency builds trust and accountability.
Interpretable models allow stakeholders to understand why decisions are made.

We inspect model coefficients as a simple explainability method.


In [8]:
coef_df = pd.DataFrame({
    "feature": X.columns,
    "coefficient": model.coef_[0]
})

coef_df


,feature,coefficient
0,experience,0.551785
1,education,0.435467
2,test_score,0.092027
3,gender,0.277839


### Responsible AI Frameworks

This simulation connects directly to:
- Google's AI Principles: fairness, accountability, transparency
- Microsoft’s AI for Good: responsible deployment and governance

Ethical AI requires:
- Monitoring bias
- Transparent decision-making
- Accountability mechanisms


### Summary

This notebook demonstrated how:
- Bias can be encoded in data and models
- Fairness metrics reveal hidden disparities
- Transparency improves ethical decision-making
- Responsible AI requires ongoing evaluation, not one-time fixes


### Multiple-Choice Questions

1. In this simulation, what was the primary source of bias in the hiring system?
A. Random variation in test scores  
B. Small sample size  
C. Explicit inclusion and weighting of gender in the outcome generation  
D. Overfitting due to model complexity  

2. Why is overall model accuracy insufficient for evaluating ethical AI systems?
A. Accuracy ignores computational cost  
B. Accuracy does not account for unequal treatment across groups  
C. Accuracy is unstable across datasets  
D. Accuracy is only useful for regression models  

3. What does demographic parity measure in this notebook?
A. Whether predicted outcomes match true outcomes  
B. Whether selection rates are equal across demographic groups  
C. Whether features are normally distributed  
D. Whether the model is interpretable  

4. What fairness issue is captured by differences in true positive rates between groups?
A. Demographic parity  
B. Model calibration  
C. Equal opportunity  
D. Privacy leakage  

5. Why does examining model coefficients support explainable and responsible AI?
A. It improves prediction accuracy  
B. It reveals how each feature influences decisions  
C. It eliminates bias automatically  
D. It guarantees legal compliance  
